Principal Component Analysis

Review Questions/exercises 

In [1]:
import numpy as np
import sklearn.decomposition as skd
import pandas as pd
import os
from sklearn.model_selection import KFold
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error


Pricipal Components to Reduce Dimensionality

1.  Create some random data X where you adjust correlation.  You may use numpy.random.multivariate_normal for that purpose.  (Note: you may find it hard to handle more than 3 variables in this way).  Next, generate random X that depends on all the Y components and a noise term (epsilon).  You may generate it as something like: Y = 1 + np.matmult(X, np.array([2, 3, 4])) + eps. 



In [2]:
N = 100
beta = np.array([1,2,3,4]).T
s12 = 0.9
Sigma = np.matrix([[1, s12, 0.2],\
                   [s12, 1, 0.3],\
                   [0.2, 0.3, 1]])
X = np.random.multivariate_normal(np.zeros(3), Sigma, N)
eps = np.random.normal(size=N)
X1 = np.hstack((np.ones((N,1)), X))
Y = np.matmul(X1, beta) + eps
data = pd.DataFrame(X)\
         .rename(columns={ 0: "X1", 1 : "X2", 2 : "X3"})
data["Y"] = Y
print data.head(4)

SyntaxError: invalid syntax (<ipython-input-2-d202e0f2ac9e>, line 11)

2. Estimate linear regression models where you include all combinations of the features you generated above.  Cross-validate your models.  Which ones are the best?  By how much?  (in terms of RMSE)

In [89]:
kf = KFold(n_splits=10)
rmse_dict = {}
formulas = ["Y~X1", "Y~X2", "Y~X3", "Y~ X2 + X3", "Y~ X1 + X3", "Y~ X1 + X2", "Y~ X1 + X2 + X3" ]
    
def rmse(formula):
    model = smf.ols(formula=formula, data=data_tr).fit()
    Y_test = data_test["Y"]
    hatY = model.predict(data_test)
    return np.sqrt(mean_squared_error(Y_test, hatY))

for train, test in kf.split(data):
    data_tr = data.iloc[train]
    data_test = data.iloc[test]
    for i, f in enumerate(formulas):
        try:
            rmse_dict[f].append(rmse(f))
        except KeyError:
            rmse_dict[f] = [ rmse(f)]

for f in formulas:
    print f, ":\t", np.mean(rmse_dict[f])

Y~X1 :	4.91133268381
Y~X2 :	4.24756819938
Y~X3 :	4.75721688962
Y~ X2 + X3 :	1.19518786312
Y~ X1 + X3 :	1.59121982064
Y~ X1 + X2 :	4.28713063598
Y~ X1 + X2 + X3 :	0.974455269551


Now extract the principal components from the data.  You can use scikit learn PCA or eigenvalue decomposition (numpy.linal.eig).  Repeat the previous exercise with including 1, 2, and 3 rotated X-s (in the order of importance).

Be careful to a) order the components according to importance; b) use the rotation matrix in the right way.  It is a square matrix, so it is too easy to mess up the transposition of it, and your rotation will be carbage.

In [93]:
pca = skd.PCA(3)
X = data[["X1", "X2", "X3"]]
pca.fit(X)
print "components"
R = pca.components_.T
print R
print np.matrix(R)
print "explained variance"
print pca.explained_variance_
print "explained variance ratio"
print pca.explained_variance_ratio_

XR = np.matmul(X, R)
data_R = pd.DataFrame(XR)\
         .rename(columns={ 0: "R1", 1 : "R2", 2 : "R3"})
data_R["Y"] = Y
print data.head(3)
rmse_dict = {}
formulas = ["Y~R1", "Y~R2", "Y~R3", "Y~ R2 + R3", "Y~ R1 + R3", "Y~ R1 + R2", "Y~ R1 + R2 + R3" ]
    
def rmse(formula):
    model = smf.ols(formula=formula, data=data_tr).fit()
    Y_test = data_test["Y"]
    hatY = model.predict(data_test)
    return np.sqrt(mean_squared_error(Y_test, hatY))

for train, test in kf.split(data):
    data_tr = data_R.iloc[train]
    data_test = data_R.iloc[test]
    for i, f in enumerate(formulas):
        try:
            rmse_dict[f].append(rmse(f))
        except KeyError:
            rmse_dict[f] = [ rmse(f)]

print
for f in formulas:
    print f, ":\t", np.mean(rmse_dict[f])



Y~R1 :	2.25254738237
Y~R2 :	7.28308014861
Y~R3 :	7.44509117451
Y~ R2 + R3 :	7.35335512054
Y~ R1 + R3 :	2.2687330243
Y~ R1 + R2 :	0.994970873349
Y~ R1 + R2 + R3 :	0.974455269551
components
[[ 0.61800364 -0.36479113 -0.69641865]
 [ 0.64887895 -0.26346488  0.71382236]
 [ 0.44387792  0.89303622 -0.07388302]]
[[ 0.61800364 -0.36479113 -0.69641865]
 [ 0.64887895 -0.26346488  0.71382236]
 [ 0.44387792  0.89303622 -0.07388302]]
explained variance
[ 2.06893512  1.04695634  0.07981094]
explained variance ratio
[ 0.6474117   0.32761384  0.02497446]
         X1        X2        X3         Y
0 -0.287501 -0.235216  0.690558  2.601684
1 -1.248377 -1.380478  0.654103 -2.030550
2  2.007265  1.159476 -0.171720  9.050766


What do you find?  Note: you should see that first 1-2 components will matter most for RMSE, the latter one is of little importance.

Change correlations at the data generation at the top of the exercise and repeat everything.  How does this influence the RMSE-s?  Can you explain what's going on?
